# TODO
Per ogni task (gap filling e reconstruction):

    - Tabella win tie losses per modello (6, 3) => Model Comparison
    - Risultato wilcoxon per tipo di dati di training => Training Data Comparison
    - Tabella win tie loss per modello per dati di training (12, 3) => Model-Training Data Comparison

In [1]:
import os
import pandas as pd
from pprint import pprint
from scipy.stats import wilcoxon
from itertools import combinations, product
from sanitize_ml_labels import sanitize_ml_labels

In [2]:
RESULT_PATH = "./wilcoxon/"
os.makedirs(RESULT_PATH, exist_ok=True)

# Helper

In [3]:
squared = lambda x: (y for y in product(list(x), list(x)) if y[0] != y[1])

In [4]:
accuracy = lambda x: x.Accuracy.values
auprc = lambda x: x.AUPRC.values
auroc = lambda x: x.AUROC.values
metrics = {
    sanitize_ml_labels("accuracy"):accuracy,
    sanitize_ml_labels("auprc"):auprc,
    sanitize_ml_labels("auroc"):auroc
}

In [5]:
def compare_2_models(d1, d2, extract_metric, p_treshold = 0.05):
    x = extract_metric(d1)
    y = extract_metric(d2)
    diff = x - y
    if diff.sum() == 0:
        return {
        "win":0,
        "tie":1,
        "losses":0,
        "pvalue":0,
        }
    stats, pvalue = wilcoxon(x, y)
    if pvalue <= p_treshold:
        if (diff > 0).mean() > 0.5:
            win = 1
            lose = 0
        else:
            win = 0
            lose = 1
        tie = 0
    else:
        tie = 1
        win = 0
        lose = 0
    return {
        "win":win,
        "tie":tie,
        "losses":lose,
        "pvalue":pvalue,
    }

In [6]:
def sanitize_df(df):
    df.columns = sanitize_ml_labels(df.columns)
    df.index.names = list(
        map(lambda x: x[:-1] if x[-1].isdigit() else x , 
            sanitize_ml_labels(df.index.names)
        )
    )
    for col in df.columns[df.dtypes == object]:
        df[col] = sanitize_ml_labels(df[col])
        
    return df

# Tabella win tie loss per modello per dati di training (12, 3)

In [7]:
def model_tables(df, task, name="model_comparison"):
    combined_group = {
        key:val
        for key, val in df.groupby(["Model", "Trained on"])
    }
    res = pd.DataFrame([
        {
            "metric":metric,
            "model1":m1,
            "train1":t1,
            "model2":m2,
            "train2":t2,
            **compare_2_models(combined_group[(m1, t1)], combined_group[(m2, t2)], metric_function),
        }
        for metric, metric_function in metrics.items()
        for (m1, t1), (m2, t2) in squared(list(product(set(df.Model), set(df["Trained on"]))))
    ])
    for metric in metrics.keys():
        r = res[res.metric == metric].drop(columns="pvalue").groupby(["train1", "model1"]).sum()
        r = sanitize_df(r)
        r.to_csv(RESULT_PATH+name+"_%s_%s.csv"%(task, metric))
        with open(RESULT_PATH+name+"_%s_%s.tex"%(task, metric), "w") as f:
            r.columns = [
                "\\textbf{%s}"%x
                for x in r.columns
            ]
            r.to_latex(
                f,
                index=False,
                caption=f"Win-Tie-Losses table for {sanitize_ml_labels(task)} obtained from Wilcoxon signed-rank test on {metric}.",
                label=f"tab:{task}_{name}_{metric}",
                column_format="|{}|".format(
                        "|".join("c" * len(r.columns))
                    ),
                escape=False,
                )

# Risultato wilcoxon per tipo di dati di training

In [8]:
def train_tables(df, task, name="training_data_comparison"):
    train_groupby = {
        key:val
        for key, val in df.groupby(["Trained on"])
    }
    res = pd.DataFrame([
        {
            "metric":metric,
            "train1":t1,
            "train2":t2,
            **compare_2_models(train_groupby[t1], train_groupby[t2], metric_function),
        }
        for metric, metric_function in metrics.items()
        for t1, t2 in combinations(set(df["Trained on"]), 2)

    ])
    res = sanitize_df(res)
    res = res.set_index("Metric")
    res.to_csv(RESULT_PATH+name+"_%s.csv"%task)
    display(res)
    with open(RESULT_PATH+name+"_%s.tex"%(task), "w") as f:
        res.columns = [
                "\\textbf{%s}"%x
                for x in res.columns
            ]
        res.to_latex(
            f,
            index=False,
            caption=f"Win-Tie-Losses table for {sanitize_ml_labels(task)} obtained from Wilcoxon signed-rank test.",
            label=f"tab:{task}_{name}",
            column_format="|{}|".format(
                    "|".join("c" * len(res.columns))
                ),
            escape=False,
            )

# Tabella win tie losses per modello (6, 3)

In [9]:
def model_total_tables(df, task, name="model_training_data_comparison"):
    model_groupby = {
        key:val
        for key, val in df.groupby(["Model"])
    }
    res = pd.DataFrame([
        {
            "metric":metric,
            "model1":m1,
            "model2":m2,
            **compare_2_models(model_groupby[m1], model_groupby[m2], metric_function),
        }
        for metric, metric_function in metrics.items()
        for m1, m2 in squared(df.Model.unique())

    ])
    for metric in metrics.keys():
        r = res[res.metric == metric].drop(columns="pvalue").groupby(["model1"]).sum()
        r = sanitize_df(r)
        r = r.reset_index()
        r.to_csv(RESULT_PATH+name+"_%s_%s.csv"%(task, metric))
        display(r)
        with open(RESULT_PATH+name+"_%s_%s.tex"%(task, metric), "w") as f:
            r.columns = [
                "\\textbf{%s}"%x
                for x in r.columns
            ]
            r.to_latex(
                f,
                index=False,
                caption=f"Win-Tie-Losses table for {sanitize_ml_labels(task)} obtained from Wilcoxon signed-rank test on {metric}.",
                label=f"tab:{task}_{name}_{metric}",
                column_format="|{}|".format(
                        "|".join("c" * len(r.columns))
                    ),
                escape=False,
                )

# Gap filling

In [10]:
def get_gap_filling_data(path="./reports/"):
    df = pd.concat([
        pd.read_csv(path+file, index_col=0)
        for file in os.listdir(path)
    ])
    df = df[df.task == "gap_filling"]
    df = df.drop("dataset", axis=1)
    df = df[df.run_type != "biological validation"]
    df = sanitize_df(df)
    return df

In [11]:
df = get_gap_filling_data()
model_tables(df, "gap_filling")
train_tables(df, "gap_filling")
model_total_tables(df, "gap_filling")

,Train1,Train2,Win,Tie,Losses,Pvalue
Metric,,,,,,
Accuracy,Multivariate gaps,Single gap,1,0,0,0.000002
AUPRC,Multivariate gaps,Single gap,1,0,0,0.000292
AUROC,Multivariate gaps,Single gap,1,0,0,0.000130


,Model,Win,Tie,Losses
0,CAE 1000,2,1,2
1,CAE 200,0,1,4
2,CAE 500,0,1,4
3,CNN 1000,3,2,0
4,CNN 200,2,3,0
5,CNN 500,3,2,0


,Model,Win,Tie,Losses
0,CAE 1000,2,0,3
1,CAE 200,1,0,4
2,CAE 500,0,0,5
3,CNN 1000,3,2,0
4,CNN 200,3,2,0
5,CNN 500,3,2,0


,Model,Win,Tie,Losses
0,CAE 1000,2,0,3
1,CAE 200,1,0,4
2,CAE 500,0,0,5
3,CNN 1000,3,2,0
4,CNN 200,3,1,1
5,CNN 500,4,1,0


# Reconstruction

In [12]:
def get_reconstruction_data(path="./reports/"):
    df = pd.concat([
        pd.read_csv(path+file, index_col=0)
        for file in os.listdir(path)
    ])
    df = df[df.task == "reconstruction"]
    df = df.drop("dataset", axis=1)
    df = df[df.run_type != "biological validation"]
    df = sanitize_df(df)
    return df

In [13]:
df = get_reconstruction_data()

In [14]:
model_tables(df, "reconstruction")
train_tables(df, "reconstruction")
model_total_tables(df, "reconstruction")

,Train1,Train2,Win,Tie,Losses,Pvalue
Metric,,,,,,
Accuracy,Multivariate gaps,Single gap,0,1,0,0.517101
AUPRC,Multivariate gaps,Single gap,0,1,0,0.461633
AUROC,Multivariate gaps,Single gap,0,1,0,0.409656


,Model,Win,Tie,Losses
0,CAE 1000,0,0,2
1,CAE 200,2,0,0
2,CAE 500,1,0,1


,Model,Win,Tie,Losses
0,CAE 1000,0,0,2
1,CAE 200,2,0,0
2,CAE 500,1,0,1


,Model,Win,Tie,Losses
0,CAE 1000,0,0,2
1,CAE 200,2,0,0
2,CAE 500,1,0,1
